# Radial Basis Function Networks
In this Notebook we will implement and test a simple RBFN. As always, we will start by defining the model class. To create a RBFN, we need to store the following parameters:
* Centers and spread of the Radial Basis Functions $\mathbf{t} \in \mathbb{R}^{(h \times i)}$, $\sigma \in \mathbb{R}$
* Weights between the hidden layer and the output layer $W \in \mathbb{R}^{(o \times h)}$

(where $o, i$ and $h$ are respectively the number of *input* units, *output* units and the number of *hidden* units)

We also define $\varphi$ (which is the RBF that will be used by the hidden units) as the *Gaussian*.

In [79]:
import numpy as np
import matplotlib.pyplot as plt 

# Model class
class RBFN:
    # We must define the number of input units, the number of hidden units and the number of output units
    def __init__(self, h):
        self.h = h
        self.trained = False
    
    def gaussian(d, sigma):
        return exp(-d**2/(2*sigma)**2)

    # Given a training set, determine the centers of the RBFs by mans of clustering
    def initialize_centers(self, X, eta = 0.1, epochs = 1):
        # Assign the centers randomly
        self.t = np.random.rand(self.h, X.shape[1])
        # An epoch defines an entire run of the dataset
        for e in range(epochs):
            for x_i in X:
                # Get the closest center to x_i
                dist = np.linalg.norm(x_i - self.t, axis=-1) 
                cmin = np.argmin(dist)
                # Apply the correction
                if(dist[cmin] != 0): self.t[cmin] += eta * (x_i - self.t[cmin])

    # Initialize Sigma by normalization
    def initialize_sigma(self):
        # Compute pairwise distances, then take the maximum
        p_dists = np.linalg.norm(self.t[None, :] - self.t[:, None], axis =-1)
        d_max = np.max(p_dists)
        # initialize sigma
        self.sigma = d_max**2 / np.sqrt(self.h)

    # Training the network involves 3 main steps:
    # * Initialize centers by means of clustering
    # * Initialize sigma
    # * Compute the weights with the pseudoinverse method
    def train(self, X, y):
        self.initialize_centers(X)
        self.initialize_sigma()


X = np.random.rand(50, 2)

rbfn = RBFN(2)
rbfn.initialize_centers(X)

# plt.scatter(X[:, 0], X[:, 1])
# plt.scatter(rbfn.t[:, 0], rbfn.t[:, 1])
rbfn.initialize_sigma()